In [3]:
# Need this to access files from google drive

from google.colab import drive

drive.mount('/content/gdrive/2023\ Sem1/COMPSCI\ 361/361\ Group\ Work')

ValueError: ignored

# COMPSCI 361: Machine Learning
# Assignment 3: Group Project


# **Task 1: Exploratory Data Analytics**

In [ ]:
"""
 I think this part and the vectorization should only happen once in Task 1.
 It's mentioned in the pdf instructions.
""" 

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load the dataset
data = pd.read_csv('train.csv')
data.head()

FileNotFoundError: ignored

# **Task 2: Classification Models Learning**

## **Part a - NB:**
Train a Naive Bayes classifier using all articles features. Report the \
(i) top-20 most identifiable words that are most likely to occur in the articles over two classes using your NB classifier and  \
(ii) the top-20 words that maximize the following quantity: \
𝑷(𝑿_𝒘 = 𝟏|𝒀 = 𝒚) / 𝑷(𝑿_𝒘 = 𝟏|𝒀 ≠ 𝒚) \
Which list of words describe the two classes better? Briefly explain your reasoning.

In [ ]:
# Create the feature matrix using TF-IDF vectorization - encode document
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Text'])

# encode document
vector = vectorizer.transform(data['Text'])

print(f'features\n {vectorizer.get_feature_names_out()}\n')
# summarize encoded vector
print(f'vector shape: {vector.shape}\n')
print(f'article vector\n {vector.toarray()}')

Feature Extraction (Vectorization) using Term Frequency - Inverse Document Frequency (TF-IDF)

In [ ]:
# Create the target variable
y = data['Category']

# Train the Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X, y)

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Top-20 Most Identifiable Words
class_labels = classifier.classes_
top_identifiable_words = []

for i, class_label in enumerate(class_labels):
    class_prob_sorted = classifier.feature_log_prob_[i, :].argsort()
    top_words = [feature_names[word_index] for word_index in class_prob_sorted[:-21:-1]]
    top_identifiable_words.append(top_words)

print("Top-20 Most Identifiable Words:")
for i, class_label in enumerate(class_labels):
    print(f"Class: {class_label}")
    print(', '.join(top_identifiable_words[i]))

# Top-20 Words Maximizing the Quantity
word_quantity = {}

print("\nTop-20 Words Maximizing the Quantity:")
for i, class_label in enumerate(class_labels):
    for word_index in range(len(feature_names)):
        quantity = classifier.feature_log_prob_[i, word_index] - classifier.feature_log_prob_[(i+1) % len(class_labels), word_index]
        word_quantity[feature_names[word_index]] = quantity

    top_quantity_words = sorted(word_quantity, key=word_quantity.get, reverse=True)[:20]
    print(f"Class: {class_label}")
    print(', '.join(top_quantity_words))

**2.a(i)**
The top-20 most identifiable words that are most likely to occur in the articles over two classes using our NB classifier are:
\
Class: entertainment 
>    film, best, said, show, band, music, year, awards, us, award, actor, album, star, chart, tv, also, number, oscar, top, new 

Class: tech 
>   said, people, mobile, software, games, phone, net, users, technology, mr, microsoft, virus, computer, broadband, new, use, could, would, digital, game \


**2.a(ii)**
The top-20 words that maximize the following quantity, 𝑷(𝑿_𝒘 = 𝟏|𝒀 = 𝒚) / 𝑷(𝑿_𝒘 = 𝟏|𝒀 ≠ 𝒚), are: 

Class: entertainment
>    film, band, best, actor, album, chart, oscar, singer, award, actress, star, musical, stars, festival, comedy, awards, aviator, theatre, rock, nominated

Class: tech
>    mobile, software, users, microsoft, games, net, technology, virus, phone, broadband, computer, phones, spam, mail, firms, use, spyware, online, pc, internet


** Which list of words describe the two classes better? Briefly explain your reasoning ** \
By maximizing 𝑷(𝑿_𝒘 = 𝟏|𝒀 = 𝒚) / 𝑷(𝑿_𝒘 = 𝟏|𝒀 ≠ 𝒚), the list focuses on words that have a greater chance of being in one class than another. In this list, the selected words mobile, software, technology, etc., are highly specific to the tech class. These words are directly associated with technology and have a higher chance of appearing in articles from the tech class. 
Whereas, the "Top-20 Most Identifiable Words" list includes words like 'us' and 'also' that are not as specific to a particular class. These words may occur frequently in either class, have a more general usage across many domains, and may not be as descriptive when it comes to class separation.

Thus, the "Top-20 Words Maximizing the Quantity" list is better for capturing the specific and discriminative words that are more indicative of the class, while the "Top-20 Most Identifiable Words" list includes words that have less class specific information or have a general usage across different domains.



## **Part b - SVM**

## **Part d - NN:**
Consider a neural network with the following hyperparameters: the initial weights uniformly drawn in range [0,0.1] with learning rate 0.01.

(i) Train a single hidden layer neural network using the hyperparameters on the training 
dataset, except for the number of hidden units (x) which should vary among 5, 20, and 40. Run the optimization for 100 epochs each time. Namely, the input layer consists of n features x = [x1, ..., xn]T , the hidden layer has x nodes z = [z1, ..., zx]T , and the output 
layer is a probability distribution y = [y1, y2]T over two classes.

(ii) Plot the average training cross-entropy loss as shown below on the y-axis versus the number of hidden units on the x-axis. Explain the effect of numbers of hidden units. 


In [ ]:
"""Examine the data"""
import pandas as pd

df = pd.read_csv('train.csv')
df.head()
df.info()
df = df.drop_duplicates()
df.groupby('Category').describe()

In [ ]:
"""Load data from train.csv and test.csv"""
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
df = df.drop_duplicates()
df_test = pd.read_csv('test.csv')

In [ ]:
"""Preprocess data"""
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical

def remove_stop_words(text):
    text = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)
#train.csv preprocessing:
x = df.apply(lambda row: remove_stop_words(row['Text']), axis=1)
y = df['Category']

label_encoder = LabelEncoder() # Encode categories into numeric values
y = label_encoder.fit_transform(y)

max_words = 20000
max_length = 500

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
x = pad_sequences(sequences, maxlen=max_length)

num_classes = len(df['Category'].unique())
y = to_categorical(y, num_classes=num_classes) # Convert y to one-hot encoded format

#test.csv preprocessing:
x_test = df_test.apply(lambda row: remove_stop_words(row['Text']), axis=1)

sequences_test = tokenizer.texts_to_sequences(x_test) # Tokenize and pad sequences
x_test = pad_sequences(sequences_test, maxlen=max_length)

y_test = df_test['Category']
y_test = label_encoder.transform(y_test)
y_test = to_categorical(y_test, num_classes=num_classes)

**2.d(i)** Train a single hidden layer neural network using the hyperparameters on the training 
dataset, except for the number of hidden units (x) which should vary among 5, 20, and 40. Run the optimization for 100 epochs each time. Namely, the input layer consists of n features x = [x1, ..., xn]T , the hidden layer has x nodes z = [z1, ..., zx]T , and the output 
layer is a probability distribution y = [y1, y2]T over two classes.

In [ ]:
"""Train model with the following hyper parameters:
1. initial weights uniformly drawn in range [0,0.1] 
2. earning rate 0.01
3. 100 epoch
3. loss function: loss = -y1*log(y_pred1) - y2*log(y_pred2)"""
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.initializers import RandomUniform

#Define loss function
def custom_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    loss = -K.sum(y_true * K.log(y_pred), axis=1)
    return loss

model = Sequential()
learning_rate = 0.01  # Set the desired learning rate = 0.01
optimizer = Adam(learning_rate=learning_rate)
weight_init = RandomUniform(minval=0, maxval=0.1) #Set weight intervals to [0,0.1]
model.add(Embedding(max_words, 32, input_length=max_length))
model.add(Flatten())
model.add(Dense(128, activation='relu'))#, kernel_initializer=weight_init))  #After asking Thomas, he said it's okay to leave the random weights initialisations on [0,0.1] out
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=custom_loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

hist = model.fit(x, y, validation_split=0.2, epochs=100, batch_size=100) #epochs=100

In [ ]:
"""Plot training and validation accuracy graph"""
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

acc = hist.history['accuracy']
val = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, '-', label='Training accuracy')
plt.plot(epochs, val, ':', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()

**2.d(ii)** Plot the average training cross-entropy loss as shown below on the y-axis versus the number of hidden units on the x-axis. Explain the effect of numbers of hidden units. 


In [ ]:
"""Plot the average training cross-entropy loss as shown below on the y-axis versus the 
number of hidden units on the x-axis"""
import matplotlib.pyplot as plt
import numpy as np

hidden_units = [5, 20, 40]
losses = []

for units in hidden_units:
    # Retrain the model
    model = Sequential()
    learning_rate = 0.01  # Set the desired learning rate = 0.01
    optimizer = Adam(learning_rate=learning_rate)
    #weight_init = RandomUniform(minval=0, maxval=0.1) #Set weight intervals to [0,0.1]
    model.add(Embedding(max_words, 32, input_length=max_length))
    model.add(Flatten())
    model.add(Dense(units, activation='relu'))#, kernel_initializer=weight_init)) #After asking Thomas, he said it's okay to leave the random weights initialisations on [0,0.1] out
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=custom_loss, optimizer=optimizer, metrics=['accuracy'])
    # Train the model
    hist = model.fit(x, y, validation_split=0.2, epochs=100, batch_size=50, verbose=0)
    
    # Compute the average training cross-entropy loss
    avg_loss = np.mean(hist.history['loss'])
    losses.append(avg_loss)

# Plot the results
plt.plot(hidden_units, losses, marker='o')
plt.xlabel('Number of Hidden Units')
plt.ylabel('Average Training Cross-Entropy Loss')
plt.title('Effect of Number of Hidden Units on Loss')
plt.grid(True)
plt.show()


**Explanation of the above graph:**
The number of hidden units in a neural network refers to the number of neurons or units in the hidden layer(s) of the network. The average training cross-entropy loss measures how well the network is able to minimize the difference between predicted and actual output during training.

The general intuition behind the relationship between the number of hidden units and average training cross-entropy loss is as follows:

1. Fewer hidden units: If the network has a small number of hidden units, it may have limited capacity to capture the complexity of the underlying data. In this case, the network may struggle to learn the patterns and relationships in the data, leading to a higher training loss. The model may underfit the data, resulting in lower accuracy and higher loss values.

2. Optimal number of hidden units: There is typically an optimal number of hidden units that allows the network to effectively learn the patterns in the data without overfitting. This optimal number can vary depending on the specific problem and dataset. Increasing the number of hidden units beyond this optimal point may not provide significant improvement in performance and can potentially lead to overfitting.

3. Too many hidden units: If the network has an excessively large number of hidden units, it may have excessive capacity and may start to memorize the training examples instead of learning the underlying patterns. This can lead to overfitting, where the network performs well on the training data but fails to generalize to unseen data. In such cases, the training loss may be very low, indicating good performance on the training set, but the model may perform poorly on new data.

Everytime we re-run the above code, it outputs a different graph, sometimes shows that the average accuracy decreases as number of hidden units increases, soemtimes shows that the average accuracy decreases from increasing 5 hidden units to 20 but increases from 20 to 40 hidden units. And there are other results as well.

This depends on several parameters in the trained model, especially when initial weights is up to random in intervals [0, 0.1].

# **Task 3: Classification Quality Evaluation**



## **Part a:**
We explore how the size of the training data set affects the test and train accuracy. For each value of m in [0.1, 0.3, 0.5, 0.7, 0.9], train your classifier on the first m portion of the training examples (that is, use the data given by XTrain[0:mN] and yTrain[0:mN]). \
\
Please report two plots: \
(i) training and \
(ii) testing accuracy for each such value of m with the x-axis referring to m
and the y-axis referring to the classification accuracy in 𝐹𝐹1 measure as shown below. In total,
there should be four curves for training accuracy and four curves for testing accuracy. \
Explain the general trend of the two plots in terms of training and testing accuracy if any. \

𝐹1 = 2 ∙ ( (𝑃recision × 𝑅ecall) / (𝑃recision + 𝑅ecall) ) \

### **1. NB:**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

# Load the training dataset
train_data = pd.read_csv('train.csv')

# Load the testing dataset
test_data = pd.read_csv('test.csv')

# Create the feature matrix using TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['Text'])
X_test = vectorizer.transform(test_data['Text'])

# Create the target variables
y_train = train_data['Category']
y_test = test_data['Category']

# Define the values of m
m_values = [0.1, 0.3, 0.5, 0.7, 0.9]

# Initialize lists to store results
train_accuracies = []
test_accuracies = []

# Perform m-based training and evaluation
for m in m_values:
    # Calculate the number of examples based on m
    N = int(X_train.shape[0] * m)
    
    # Select the first N examples for training
    X_train_subset, y_train_subset = X_train[:N], y_train[:N]
    
    # Train the Naive Bayes classifier
    classifier = MultinomialNB()
    classifier.fit(X_train_subset, y_train_subset)
    
    # Predict on training and testing data
    y_train_pred = classifier.predict(X_train_subset)
    y_test_pred = classifier.predict(X_test)
    
    # Calculate training and testing accuracy using F1 score
    train_accuracy = f1_score(y_train_subset, y_train_pred, average='weighted')
    test_accuracy = f1_score(y_test, y_test_pred, average='weighted')
    
    # Store accuracy values
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    
    # Print the evaluation metrics for each m
    print(f"Training Data Siz (m) = {m}")
    print("Training Accuracy (F1-score):", train_accuracy)
    print("Testing Accuracy (F1-score):", test_accuracy)
    print("--------------------------------------")


In [ ]:
import matplotlib.pyplot as plt

# Plot the training and testing accuracy curves
plt.plot(m_values, train_accuracies, marker='o', label="Training Accuracy")
plt.plot(m_values, test_accuracies, marker='o', label="Testing Accuracy")
plt.xlabel("Training Data Size (m)")
plt.ylabel("Accuracy (F1-score)")
plt.title("Training and Testing Accuracy vs. Training Data Size")
plt.legend()
plt.grid(True)
plt.show()

** Explain the general trend of the two plots in terms of training and testing accuracy if any. **

The training accuracy is consistently high and close to 1 across all the different sizes of the training data evaluated. As more training data is included, the classifier becomes more capable of capturing the underlying patterns, resulting in higher accuracy.

The testing accuracy shows a noticeable difference when evaluated on a very small subset of the training data (m = 0.1). In this case, the testing accuracy is significantly lower than the training accuracy. This suggests that the classifier may be overfitting to the small training subset and doesn't generalize well to new data. However, as the size of the training data increases (m = 0.3, 0.5, 0.7, 0.9), both the training and testing accuracies are high and align closely. 

The general trend suggests that increasing the size of the training data improves the classifier's ability to generalize and perform well on new data and reduces the risk of overfitting.

### **4. NN:**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from keras.optimizers import Adam

def remove_stop_words(text):
    text = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)

df = pd.read_csv('train.csv')
df = df.drop_duplicates()

x = df.apply(lambda row: remove_stop_words(row['Text']), axis=1)
y = df['Category']

# Encode categories into numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical

max_words = 20000
max_length = 500

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
x = pad_sequences(sequences, maxlen=max_length)

# Convert y to one-hot encoded format
num_classes = len(df['Category'].unique())
y = to_categorical(y, num_classes=num_classes)

df_test = pd.read_csv('test.csv')

x_test = df_test.apply(lambda row: remove_stop_words(row['Text']), axis=1)

# Tokenize and pad sequences
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences_test, maxlen=max_length)

y_test = df_test['Category']
y_test = label_encoder.transform(y_test)
y_test = to_categorical(y_test, num_classes=num_classes)

m_values = [0.1, 0.3, 0.5, 0.7, 0.9]
train_accuracies = []
test_accuracies = []

for m in m_values:
    # Determine the number of training samples based on m
    num_samples = int(m * x.shape[0])
    x_train_subset = x[:num_samples]
    y_train_subset = y[:num_samples]
    
    # Create and train the model
    model = Sequential()
    model.add(Embedding(max_words, 32, input_length=max_length))
    model.add(Flatten())
    learning_rate = 0.01  # Set the desired learning rate = 0.01
    optimizer = Adam(learning_rate=learning_rate)
    model.add(Dense(40, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=custom_loss, optimizer=optimizer, metrics=['accuracy'])
    model.fit(x_train_subset, y_train_subset, epochs=100, batch_size=100, verbose=0)
    
    # Predict on the training and test datasets
    y_train_pred = model.predict(x_train_subset)
    y_test_pred = model.predict(x_test)
    
    # Convert one-hot encoded labels to categorical labels
    y_train_subset_labels = np.argmax(y_train_subset, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    
    # Convert predictions to class labels
    y_train_pred_labels = np.argmax(y_train_pred, axis=1)
    y_test_pred_labels = np.argmax(y_test_pred, axis=1)
    
    # Compute accuracy using F1 measure
    train_accuracy = f1_score(y_train_subset_labels, y_train_pred_labels, average='micro')
    test_accuracy = f1_score(y_test_labels, y_test_pred_labels, average='micro')
    
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Plot the training and testing accuracies
plt.plot(m_values, train_accuracies, marker='o', label='Training Accuracy')
plt.plot(m_values, test_accuracies, marker='o', label='Testing Accuracy')
plt.xlabel('Training Data Size (m)')
plt.ylabel('Accuracy (F1 Measure)')
plt.title('Effect of Training Data Size on Accuracy')
plt.legend()
plt.grid(True)
plt.show()


Note that due to random initial weights, the model generates different results everytime it's reran.

In general, as the size of the training dataset (m) increases, we can expect the following trends:

**Training Accuracy:**

Initially, when the training dataset is small (m=0.1), the model has limited exposure to the data, resulting in lower training accuracy. The model may struggle to capture the underlying patterns and generalize well.

As the training dataset size increases, the model has access to more diverse examples and can learn more representative patterns. Consequently, the training accuracy tends to improve.

However, there might be a saturation point where further increasing the training dataset size doesn't significantly improve the training accuracy. This could indicate that the model has already captured the essential patterns and additional data doesn't contribute significantly to its performance.

**Testing Accuracy:**

Similar to training accuracy, when the training dataset is small (m=0.1), the model may have limited exposure to the data, leading to lower testing accuracy. The model might not generalize well to unseen examples.

As the training dataset size increases, the model has more examples to learn from and can potentially generalize better to unseen data. This often results in an improvement in testing accuracy.

However, it's important to note that if the training dataset becomes too large (m=0.9), there might be a risk of overfitting. Overfitting occurs when the model becomes too specific to the training data and performs poorly on unseen examples. This can lead to a decrease in testing accuracy despite high training accuracy.

In summary, the general trend in the two plots of training and testing accuracy as the training dataset size increases is an initial improvement in accuracy as the model gets exposed to more data and learns representative patterns. However, there may be a point of diminishing returns or a risk of overfitting if the dataset size becomes excessively large. It's crucial to strike a balance and find the optimal dataset size that maximizes the model's ability to generalize to unseen examples.

## **Part b** 
Let’s use 5-fold cross-validation to assess model performance. Investigate the impact of key
hyperparameters of your choices for each classifier using a testing dataset. E.g., for SVM, the
classification accuracy may be significantly affected by the kernels and hyperparameter
combination. List hyperparameters for each classifier and demonstrate how these
hyperparameters impact on the testing accuracy. \


### **1. NB:**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

# Load the training dataset
train_data = pd.read_csv('train.csv')

# Load the testing dataset
test_data = pd.read_csv('test.csv')

# Create the feature matrix using TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['Text'])
X_test = vectorizer.transform(test_data['Text'])

# Create the target variables
y_train = train_data['Category']
y_test = test_data['Category']

# Define the hyperparameters to investigate
alpha_values = [0.001, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100.0]

# Initialize lists to store results
train_scores = []
val_scores = []
test_scores = []

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for alpha in alpha_values:    
    for train_index, val_index in kf.split(X_train):
        # Split the data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        
        # Train the Naive Bayes classifier
        classifier = MultinomialNB(alpha=alpha)
        classifier.fit(X_train_fold, y_train_fold)
        
        # Predict on the validation fold
        y_val_pred = classifier.predict(X_val_fold)
        
        # Calculate training and validation accuracy
        train_acc = accuracy_score(y_train_fold, classifier.predict(X_train_fold))
        val_acc = accuracy_score(y_val_fold, classifier.predict(X_val_fold))
        
        # Store the scores
        train_scores.append(train_acc)
        val_scores.append(val_acc)
    
    # Calculate average scores across the folds
    avg_train_acc = sum(train_scores) / len(train_scores)
    avg_val_acc = sum(val_scores) / len(val_scores)
    
    # Train the final classifier on the entire training set with the best hyperparameter
    final_classifier = MultinomialNB(alpha=alpha)
    final_classifier.fit(X_train, y_train)
    
    # Evaluate on the test set
    y_test_pred = final_classifier.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    # Store the scores
    train_scores.append(avg_train_acc)
    val_scores.append(avg_val_acc)
    test_scores.append(test_acc)
    
    # Print the results
    print(f"Alpha: {alpha}")
    print(f"Avg Training Accuracy: {avg_train_acc}")
    print(f"Avg Validation Accuracy: {avg_val_acc}")
    print(f"Test Accuracy: {test_acc}")
    print("-------------------------------")

# Print the overall performance summary
print("Overall Performance Summary")
print(f"Best Alpha: {alpha_values[test_scores.index(max(test_scores))]}")
print(f"Best Test F1-score: {max(test_scores)}")


In [ ]:
import matplotlib.pyplot as plt

# Plot the training and testing accuracies
plt.plot(alpha_values, train_scores[:len(alpha_values)], marker='o', label='Training Accuracy')
plt.plot(alpha_values, val_scores[:len(alpha_values)], marker='o', label='Validation Accuracy')
plt.plot(alpha_values, test_scores, marker='o', label='Testing Accuracy')
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.title('Impact of Hyperparameter Alpha on Accuracy')
plt.legend()
plt.ylim(0.9, 1.01)
plt.grid(True)
plt.show()

### **4. NN:**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Define the hyperparameters to explore
hidden_units = [5, 40]
learning_rates = [0.001, 0.1]
dropout_rates = [0.2, 0.6]
batch_sizes = [50, 100]

# Initialize lists to store results
results = []
param_combinations = []

# Perform cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for hu in hidden_units:
    for lr in learning_rates:
        for dr in dropout_rates:
            for bs in batch_sizes:
                fold_results = []
                for train_index, test_index in kf.split(x):
                    x_train, x_test = x[train_index], x[test_index]
                    y_train, y_test = y[train_index], y[test_index]

                    # Create and train the model
                    model = Sequential()
                    learning_rate = 0.01  # Set the desired learning rate = 0.01
                    optimizer = Adam(learning_rate=learning_rate)
                    weight_init = RandomUniform(minval=0, maxval=0.1) #Set weight intervals to [0,0.1]
                    model.add(Embedding(max_words, 32, input_length=max_length))
                    model.add(Flatten())
                    model.add(Dense(units, activation='relu'))#, kernel_initializer=weight_init))
                    model.add(Dense(num_classes, activation='softmax'))
                    model.compile(loss=custom_loss, optimizer=optimizer, metrics=['accuracy'])
                    # Train the model
                    hist = model.fit(x, y, validation_split=0.2, epochs=100, batch_size=50, verbose=0)

                    # Evaluate the model on testing data
                    y_pred = model.predict(x_test)
                    y_pred_labels = np.argmax(y_pred, axis=1)
                    accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_labels)
                    fold_results.append(accuracy)

                avg_accuracy = np.mean(fold_results)
                results.append(avg_accuracy)
                param_combinations.append((hu, lr, dr, bs))

# Find the best hyperparameter combination
best_index = np.argmax(results)
best_params = param_combinations[best_index]
best_accuracy = results[best_index]

# Print the best hyperparameter combination and its testing accuracy
print("Best Hyperparameters:", best_params)
print("Testing Accuracy:", best_accuracy)

# Plot the results
x_ticks = range(len(results))
plt.bar(x_ticks, results)
plt.xlabel('Hyperparameter Combinations')
plt.ylabel('Testing Accuracy')
plt.title('Impact of Hyperparameters on Testing Accuracy')
plt.xticks(x_ticks, param_combinations, rotation='vertical')
plt.tight_layout()
plt.show()



**List of hyperparameters:**

hidden_units = [5, 40]

learning_rates = [0.001, 0.1]

dropout_rates = [0.2, 0.6]

batch_sizes = [50, 100]

The affects of the above hyperparameters are displayed on the plot above, showing that different combinations of hyperparameters allows different levels of accuracy. However, it's important to note that since the initial weights are randomly assigned, everytime we rerun the model it will churn out a different optimal combinations. 

However, the general ideas are that:

1. Hidden Units: The number of hidden units determines the capacity or complexity of the model. Increasing the number of hidden units can increase the model's ability to capture complex patterns in the data, potentially leading to improved accuracy. However, using too many hidden units can also result in overfitting if the model becomes too complex for the given dataset.

2. Learning Rate: The learning rate controls the step size at which the model's weights are updated during training. A higher learning rate can result in faster convergence during training, but it may also cause the model to overshoot the optimal solution and lead to instability or divergence. On the other hand, a lower learning rate may result in slower convergence but can potentially lead to better accuracy if the model is able to find a more precise solution.

3. Dropout Rate: Dropout is a regularization technique that randomly drops out a fraction of the neurons during training. It helps prevent overfitting by reducing the interdependence of neurons and promoting better generalization. A higher dropout rate can increase the model's ability to generalize but may also lead to underfitting if too many neurons are dropped. Conversely, a lower dropout rate may result in more overfitting if the model relies heavily on specific neurons.

4. Batch Size: The batch size determines the number of samples processed before the model's weights are updated during training. A larger batch size can provide a smoother gradient estimation, leading to more stable updates and potentially faster convergence. However, using a larger batch size requires more memory, and the model may get stuck in suboptimal solutions. Smaller batch sizes can introduce more stochasticity and exploration during training, potentially leading to better accuracy, but they may also result in slower convergence.





## **Part c:**

Report and compare your NB, kNN, SVM and NN classifiers with the best hyperparameter
settings. Summarize what you have observed in the classification accuracy in 𝐹𝐹1 measure on
the testing dataset.

### **1. NB:**

### **2. kNN:**

### **3. SVM:**

### **4. NN:**

# **Task 4: Report Writing**